In [1]:
import torch
import pandas as pd

In [2]:
from transformers import pipeline, set_seed, GPT2Tokenizer, GPT2LMHeadModel
from torch import tensor, numel

In [3]:
!pip install bertviz
from bertviz import model_view

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.6/157.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.5/139.5 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [5]:
phrase = "My friend was right about this class. it is so fun!"
encoded_phrase = tokenizer(phrase, return_tensors = 'pt')

In [6]:
model = GPT2LMHeadModel.from_pretrained('gpt2')

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [8]:
response = model(**encoded_phrase, output_attentions = True, output_hidden_states = True)

`torch.nn.functional.scaled_dot_product_attention` does not support `output_attentions=True`. Falling back to eager attention. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


In [9]:
len(response.attentions)

12

In [10]:
encoded_phrase

{'input_ids': tensor([[3666, 1545,  373,  826,  546,  428, 1398,   13,  340,  318,  523, 1257,
            0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [11]:
response.attentions[-1].shape

torch.Size([1, 12, 13, 13])

In [12]:
encoded_phrase['input_ids'].shape

torch.Size([1, 13])

In [13]:
tokens = tokenizer.convert_ids_to_tokens(encoded_phrase['input_ids'][0])
tokens

['My',
 'Ġfriend',
 'Ġwas',
 'Ġright',
 'Ġabout',
 'Ġthis',
 'Ġclass',
 '.',
 'Ġit',
 'Ġis',
 'Ġso',
 'Ġfun',
 '!']

In [16]:
# Layer Index 9, head 0. Check out the almost 60% attention the token it is giving to the token class
arr = response.attentions[9][0][0]
n_digits = 3
attention_df = pd.DataFrame((torch.round(arr * 10**n_digits) / (10**n_digits)).detach()).applymap(float)
attention_df.columns = tokens
attention_df.index = tokens
attention_df

<ipython-input-16-82c90c991b0c>:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  attention_df = pd.DataFrame((torch.round(arr * 10**n_digits) / (10**n_digits)).detach()).applymap(float)


,My,Ġfriend,Ġwas,Ġright,Ġabout,Ġthis,Ġclass,.,Ġit,Ġis,Ġso,Ġfun,!
My,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Ġfriend,0.968,0.032,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Ġwas,0.824,0.145,0.031,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Ġright,0.979,0.008,0.007,0.005,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Ġabout,0.979,0.008,0.004,0.005,0.005,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Ġthis,0.924,0.031,0.007,0.006,0.016,0.016,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Ġclass,0.946,0.005,0.001,0.001,0.001,0.002,0.044,0.000,0.000,0.000,0.000,0.000,0.000
.,0.691,0.013,0.003,0.003,0.002,0.006,0.269,0.013,0.000,0.000,0.000,0.000,0.000
Ġit,0.249,0.003,0.002,0.003,0.003,0.011,0.681,0.018,0.030,0.000,0.000,0.000,0.000
Ġis,0.308,0.006,0.002,0.002,0.002,0.015,0.562,0.014,0.058,0.030,0.000,0.000,0.000


In [17]:
tokens = tokenizer.convert_ids_to_tokens(encoded_phrase['input_ids'][0])
model_view(response.attentions, tokens)

<IPython.core.display.Javascript object>

In [18]:
response.hidden_states[-1].shape

torch.Size([1, 13, 768])

In [19]:
response.logits.shape

torch.Size([1, 13, 50257])

In [20]:
pd.DataFrame(
    zip(tokens, tokenizer.convert_ids_to_tokens(response.logits.argmax(2)[0])),
    columns = ['Sequence up until', 'Next token with highest probability']
)

,Sequence up until,Next token with highest probability
0,My,Ċ
1,Ġfriend,","
2,Ġwas,Ġa
3,Ġright,.
4,Ġabout,Ġthat
5,Ġthis,.
6,Ġclass,.
7,.,ĠI
8,Ġit,Ġwas
9,Ġis,Ġa


In [22]:
generator = pipeline('text-generation', model = 'gpt2')

Device set to use cpu


In [23]:
generator(phrase, max_length = 20, num_return_sequences = 1, do_sample = False)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'My friend was right about this class. it is so fun! I love it!\n\nI'}]

In [24]:
generator(phrase, max_length = 20, num_return_sequences = 1, do_sample = True)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "My friend was right about this class. it is so fun! I love it! I can't"}]